## Transfer wiedzy

Ten notebook zawiera kod umożliwiający przeprowadzenie klasyfikacji wieloklasowej z zastosowaniem techniki transferu wiedzy.


In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

# Libraries required for the transfer learning with pre-trained models 
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.applications import ResNet50


# Libraries required for the transfer learning with pre-trained models 
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as Pre_input_MN
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as Pre_input_EF
from tensorflow.keras.applications.resnet import preprocess_input as Pre_input_RS

# data visualization and testing:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from tensorflow.keras.metrics import Precision, Recall




# DEFINICJA SŁOWNIKA Z OBIEKTAMI W ZALEŻNOŚCI OD WYKORZYSTANEJ BAZY DANYCH:

In [2]:
# We define a dictionary that can be used to map a folder name with specific class data.
# To the desired labels, we used a dumb definition: 

# FOR IMAGENETTE:
class_dictionary = {0:"gas_station", 1:	"golf_ball", 2: "radio", 3:	"chainsaw",	4: "instrument"}
def decode_predictions(array, class_dictionary):
    return np.array(list(map(class_dictionary.get, array)))


# FOR original database:
#class_dictionary = {0: "zebra", 1: "uszatek", 2: "teddy", 3: "boat", 4: "dozer", 5: "bear"} 
#def decode_predictions(array, class_dictionary):
#    return np.array(list(map(class_dictionary.get, array)))




In [3]:
# Parametrs
image_size = (224,224)   # Image size
batch_size = 32  # Batch size

# WYBÓR LICZBY KLAS W ZALEŻNOŚCI OD WYKORZYSTANEJ BAZY DANYCH

In [4]:
n_classes = 5  # Number of classes for ImageNette = 5
#n_classes = 6  # Number of classes for original database = 6

In [5]:
class_dictionary.values()

dict_values(['gas_station', 'golf_ball', 'radio', 'chainsaw', 'instrument'])

# WYBÓR BAZY DANYCH oraz LICZBY PRÓBEK

Definiowanie scieżek do bazy danej treningowej i testowej.
W celu pobrania bazy danych nalezy skorzystać z repozytorium: 
https://github.com/iitis/EduToyz




### Scenariusze testowe:
Testy zostały przeprowadzone na dwóch bazach danych: 

* ImageNette 
* autorska baza danych


W celu zbadania skalowalności techniki transferu wiedzy, modele były trenowane przy różnych liczbach próbek w zbiorze treningowym. 

Rozważano następujące scenariusze:

- Zbiór treningowy zawierający 1 obraz
- Zbiór treningowy zawierający 2 obrazy
- Zbiór treningowy zawierający 3 obrazy
- Zbiór treningowy zawierający 4 obrazy
- Zbiór treningowy zawierający 5 obrazów
- Zbiór treningowy zawierający 10 obrazów
- Zbiór treningowy zawierający 20 obrazów

Testowanie na tych różnych rozmiarach zbiorów pozwoliło na dokładną analizę wpływu liczby próbek na skuteczność klasyfikacji, 

co jest kluczowe dla oceny efektywności techniki w różnych warunkach.

## Ścieżki dla bazy ImageNette:

In [6]:
# The selection of the number of samples to create templates 
# is done manually according to the following scheme: 
# for 1 sample: Imagenette_v1/train, 
# for 2 samples: Imagenette_v2/train, 
# for 3 samples: Imagenette_v3/train,
# for 4 samples: Imagenette_v4/train, 
# for 5 samples: Imagenette_v5/train, 
# for 10 samples: Imagenette_v10/train,
# for 20 samples: Imagenette_v20/train.

path_train = "Imagenette_v1/train"


# The selection of the test set
# is done manually according to the following scheme: 
# for 1 sample: Imagenette_v1/test, 
# for 2 samples: Imagenette_v2/test, 
# for 3 samples: Imagenette_v3/test,
# for 4 samples: Imagenette_v4/test, 
# for 5 samples: Imagenette_v5/test, 
# for 10 samples: Imagenette_v10/test,
# for 20 samples: Imagenette_v20/test.


path_test = "Imagenette_v1/test"


# Ensure that the number of saplems in both selected paths to foilders always matches.


## Ścieżki dla Autorskiej bazu danych:

In [7]:
# The selection of the number of samples to create templates 
# is done manually according to the following scheme: 
# for 1 sample: Dataset_N1/train, 
# for 2 samples: Dataset_N2/train, 
# for 3 samples: Dataset_N3/train,
# for 4 samples: Dataset_N4/train, 
# for 5 samples: Dataset_N5/train, 
# for 10 samples: Dataset_N10/train,
# for 20 samples: Dataset_N20/train.

path_train = "Dataset_N1/train"

# The selection of the test set
# is done manually according to the following scheme: 
# for 1 sample: Dataset_N1/test, 
# for 2 samples: Dataset_N2/test, 
# for 3 samples: Dataset_N3/test,
# for 4 samples: Dataset_N4/test, 
# for 5 samples: Dataset_N5/test, 
# for 10 samples: Dataset_N10/test,
# for 20 samples: Dataset_N20/test.



path_test = "Dataset_N1/test"


# Ensure that the number of saplems in both selected paths to foilders always matches.

# ZBIÓR TRENINGOWY

In [ ]:
# reading data from a folder with a single example (used to create representations of categories). 
# preprocess_input is a function given by keras for a particualr model or a custiom fuction that prepares data for the model 


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_train,
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)


# BUDOWA MODELU


### Wykorzystane modele:
- MobileNetV2
- ResNet50V2
- EfficientNetV2B0


#### model_MN opiera się na MobileNetV2 - szczegóły tego modelu można znaleźć na stronie https://keras.io/api/applications/mobilenet/#mobilenetv2-function

In [ ]:
model_MN = MobileNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)
# creating a model without the top layer 
inter_output_model_MN = tf.keras.Model(inputs=model_MN.input,
                           outputs=model_MN.layers[-2].output)

#### model_EF opiera się na EfficientNetV2B0 - szczegóły tego modelu można znaleźć na stronie https://keras.io/api/applications/efficientnet_v2/#efficientnetv2b0-function

In [ ]:
model_EF = EfficientNetV2B0(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)

# creating a model without the top layer 

inter_output_model_EF = tf.keras.Model(inputs=model_EF.input,
                           outputs=model_EF.layers[-2].output)

#### model_RN opiera się na ResNetV2 - szczegóły tego modelu można znaleźć na stronie https://keras.io/api/applications/resnet/#resnet50v2-function

In [ ]:
model_RN = ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)

# creating a model without the top layer 
inter_output_model_RN = tf.keras.Model(inputs=model_RN.input, 
                            outputs=model_RN.layers[-2].output)

In [ ]:
# To create a based model, select one of the three available pre-trained models:
# MobileNetV2 > inter_output_model_MN
# EfficientNetV2B0 > inter_output_model_EF
# ResNetV2 > inter_output_model_RN

inter_output_model = inter_output_model_EF



if inter_output_model == inter_output_model_MN:
    model = "MobileNetV2" 
    normalized_ds_train = train_ds.map(lambda x, y: (Pre_input_MN(x), y))
elif inter_output_model == inter_output_model_EF:
    model = "EfficientNetV2B0" 
    normalized_ds_train = train_ds.map(lambda x, y: (Pre_input_EF(x), y))
if inter_output_model == inter_output_model_RN:
    model = "ResNetV2" 
    normalized_ds_train = train_ds.map(lambda x, y: (Pre_input_RS(x), y))


In [ ]:
Y_train = np.concatenate(np.array([y for x, y in normalized_ds_train]), axis=0) 
X_train = np.concatenate(np.array([x for x, y in normalized_ds_train]), axis=0) 

In [ ]:
# we use shuffle false to have the original order of data

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_test,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)


Found 6631 files belonging to 5 classes.


In [ ]:
if inter_output_model == inter_output_model_MN:
    normalized_ds_test = test_ds.map(lambda x, y: (Pre_input_MN(x), y))
elif inter_output_model == inter_output_model_EF:
    normalized_ds_test = test_ds.map(lambda x, y: (Pre_input_EF(x), y))
if inter_output_model == inter_output_model_RN:
    normalized_ds_test = test_ds.map(lambda x, y: (Pre_input_RS(x), y))

In [ ]:
Y_test = np.concatenate(np.array([y for x, y in normalized_ds_test]), axis=0) 
X_test = np.concatenate(np.array([x for x, y in normalized_ds_test]), axis=0) 

C:\Users\mhalama\AppData\Local\Temp\ipykernel_5596\3536204689.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y_test = np.concatenate(np.array([y for x, y in normalized_ds_test]), axis=0)
C:\Users\mhalama\AppData\Local\Temp\ipykernel_5596\3536204689.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.concatenate(np.array([x for x, y in normalized_ds_test]), axis=0)


In [ ]:
print(inter_output_model.output.shape)

(None, 1280)


# Dodanie nowych warstw dla nowego klasyfikatora

In [ ]:
# Freezing of the base model weights
inter_output_model.trainable = False

# Adding new layers for task 2
x = inter_output_model.output
x = tf.keras.layers.Dense(1024, activation='relu')(x)  # New fully bonded layer
predictions = Dense(n_classes, activation='softmax')(x) 
model = tf.keras.models.Model(inputs=inter_output_model.input, outputs=predictions)



### Ustawienia hiperparametrów:
| Hiperparametr         | Wartość                          |
|-----------------------|----------------------------------|
| Liczba epok           | 3                                |
| Optymalizator         | Adam                             |
| Funkcja straty        | categorical_crossentropy         |
| Biblioteka            | Keras                            |



In [ ]:

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])


Epoch 1/3
1/1 [==============================] - 176s 176s/step - loss: 1.5875 - accuracy: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.0663 - val_accuracy: 0.8602 - val_precision: 0.9973 - val_recall: 0.0559
Epoch 2/3
1/1 [==============================] - 145s 145s/step - loss: 0.6589 - accuracy: 1.0000 - precision: 1.0000 - recall: 0.8000 - val_loss: 0.6593 - val_accuracy: 0.9447 - val_precision: 0.9988 - val_recall: 0.6090
Epoch 3/3
1/1 [==============================] - 145s 145s/step - loss: 0.1082 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4390 - val_accuracy: 0.9606 - val_precision: 0.9974 - val_recall: 0.8174


# TRENOWANIE

In [ ]:

# Train the model according to established hyperparameters 
history = model.fit(
    train_ds,
    steps_per_epoch=len(train_ds),
    epochs=3,  # Liczba epok
    validation_data=test_ds,
    validation_steps=len(test_ds))




### Ocena modeli:

Macierz pomyłek (ang. Confusion Matrix) - przedstawia liczbę poprawnie sklasyfikowanych przykładów oraz liczby błędów

 w każdej klasie, a przedstawić to przy pomocy wzoru:

$$
A_{conf}= \begin{bmatrix}
TP & FP \\
FN & TN
\end{bmatrix}
$$

gdzie:

$TP-$ Prawdziwie Pozytywne (ang. True Positive) to liczba poprawnie sklasyfikowanych pozytywnych przykładów,

$FP-$ Fałszywie Pozytywne (ang. False Positive) to liczba błędnie sklasyfikowanych pozytywnych przykładów,

$FN-$ Fałszywie Negatywne (ang. False Negative) to liczba błędnie sklasyfikowanych negatywnych przykładów,

$TN-$ Prawdziwie Negatywne (ang. True Negative) to liczba poprawnie sklasyfikowanych negatywnych przykładów.


#### Metryki:
W celu testowania modeli zastosowano metryki takie jak 

* skuteczność:

  $$ \text{Skuteczność} = \frac{TP + TN}{TP + TN + FP + FN} $$

   Skuteczność mierzy odsetek poprawnych klasyfikacji.


* precyzja:

  $$
   \text{Precyzja} = \frac{TP}{TP + FP}
$$
   Precyzja określa, jaki odsetek przewidywanych pozytywnych przypadków jest rzeczywiście pozytywny.


* czułość:

$$
   \text{Czułość} = \frac{TP}{TP + FN}
 $$
   Czułość mierzy zdolność modelu do wykrywania rzeczywistych pozytywnych przypadków.



## Wizaualizacja wyników

In [ ]:
if path_train == "Dataset_N1/train":
    dataset_name = "autorskiej bazy danych"
else:
    dataset_name = "bazy Imagenette"

In [ ]:
print(f"Wyniki dla {dataset_name}:")
for epoch in range(3):
    print(f"Epoch {epoch + 1}/{3}")
    print(f"Training Accuracy: {history.history['accuracy'][epoch]}")
    print(f"Training Precision: {history.history['precision'][epoch]}")
    print(f"Training Recall: {history.history['recall'][epoch]}")
    print(f"Validation Accuracy: {history.history['val_accuracy'][epoch]}")
    print(f"Validation Precision: {history.history['val_precision'][epoch]}")
    print(f"Validation Recall: {history.history['val_recall'][epoch]}")
    print("-" * 30)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

custom_palette_blue = ['#24acb3','#39b9bf','#58d2d6','#78e0e3','#97edf0','#b9f8fa']
custom_palette_gray = ['#4c4c4d', '#4e4e4f', '#606061','#6f6f70', '#838385', '#9c9c9c','#cccccc','#cfd1d1', '#dee2e3']

def create_custom_cmap(colors):
    n = len(colors)
    cmap = LinearSegmentedColormap.from_list('custom_cmap', colors, N=n)
    return cmap

custom_cmap_g = create_custom_cmap(custom_palette_gray)

In [ ]:


# Obliczenia macierzy pomyłek
confusion_matrix_normalize = confusion_matrix(np.argmax(Y_test, axis=1), predictions, normalize='true')

# Rysowanie wykresu
plt.figure(figsize=(10,10))
ax = sns.heatmap(confusion_matrix_normalize, annot=True, cmap=custom_palette_gray, annot_kws={"size": 26},cbar=False)
ax.set_yticklabels(["dystrybutor \npaliwa", "piłka \n golfowa", "radio", "piła \nłańcuchowa"," waltornia"],fontsize = 20, rotation = 45)
ax.set_xticklabels(["dystrybutor \npaliwa", "piłka \n golfowa", "radio", "piła \nłańcuchowa"," waltornia"],fontsize = 20, rotation = 45)
# ax.set_yticklabels(["zebra","Uszatek", "teddy","boat", "dozer", "bear"], va='center')
# ax.set_xticklabels(["zebra","Uszatek", "teddy","boat", "dozer", "bear"])

plt.tight_layout()
plt.show()
